In [ ]:
import os
import json
from glob import glob
import numpy as np

import avstack
import avapi

path_to_results = "../../../../outputs/last_run"

In [ ]:
# Load in the metadata
with open(os.path.join(path_to_results, "launch_metadata.json"), 'r') as f:
    launch_metadata = json.load(f)
with open(os.path.join(path_to_results, "simulator", "metadata.json"), "r") as f:
    sim_metadata = json.load(f)

# Set up the carla dataset manager
CSM = avapi.carla.CarlaScenesManager(data_dir=sim_metadata["dataset_path"])
CDM = CSM.get_scene_dataset_by_index(sim_metadata["scene_idx"])
all_ts = np.array(CDM.timestamps)

# Get the frame/time mappings
with open(os.path.join(path_to_results, "simulator", "idx_to_frame_map.txt"), "r") as f:
    idx_to_frame_map = {int(line.rstrip().split()[0]): int(line.rstrip().split()[1]) for line in f.readlines()}
frame_to_idx_map = {v:k for k, v in idx_to_frame_map.items()}

# Load in the agent paths
output_paths = {}
for agent in ["ego"] + [f"agent{i}" for i in range(launch_metadata["n_infrastructure_agents"])]:
    output_paths[agent] = {}
    for subfolder in ["detections", "tracks"]:
        path_to_sub = os.path.join(path_to_results, agent, subfolder)
        files_in_folder = [os.path.basename(file) for file in sorted(glob(os.path.join(path_to_sub, "*.txt")))]
        output_paths[agent][subfolder] = {}
        for i, file in enumerate(files_in_folder):
            ts = float(file.split('-')[3].replace('.txt', ''))
            frame = CDM.frames[np.argmin(abs(all_ts - ts))]
            idx = frame_to_idx_map[frame]
            output_paths[agent][subfolder][i] = {
                "timestamp":ts,
                "idx": idx,
                "frame": frame,
                "path": os.path.join(path_to_sub, file)
            }

In [ ]:
output_paths

In [ ]:
idx_to_frame_map

In [ ]:
ts

In [ ]:
all_ts

In [ ]:
len(output_paths["ego"]["detections"])

## Visualize ego information